In [1]:

%load_ext tensorboard
%tensorboard --logdir runs

# Set YOLO settings for TensorBoard
from ultralytics import settings
settings.update({'tensorboard': True})

Reusing TensorBoard on port 6007 (pid 65756), started 2 days, 1:13:16 ago. (Use '!kill 65756' to kill it.)

In [ ]:
from ultralytics import YOLO
import os

CONFIG_YAML_FOLDER = 'dataset/training/config_yamls'
K = 4

base_model = 'yolo11'
sizes = ['n', 's', 'm', 'l'] 
models = {'_det': '', '_seg': '-seg', '_pose': '-pose'}
extensions = ['.pt'] # , '.yaml'

project_name = {'_det': 'pieces', '_seg': 'board', '_pose': 'board'}

batch_sizes = {
    'n': 8,  # Nano
    's': 2,  # Small
    'm': 1,  # Medium
    'l': 1,  # Large
}

for e in extensions:
    for m in models:
        for s in sizes:
            model_name = f"{base_model}{s}{models[m]}{e}"
            for k in range(K):
                model = YOLO(model_name)
                config_yaml = os.path.join(CONFIG_YAML_FOLDER, f'fold_{k}{m}.yaml')
                                
                model.train(
                    data = config_yaml,
                    epochs = 100,
                    time = 1, # Set max time
                    patience = 20, # Early stopping if no improvement
                    imgsz=640,
                    batch=batch_sizes[s],
                    device = '0',  # Use GPU
                    save = True,
                    save_period = 1,
                    #cache = True,
                    exist_ok = True,
                    project = f'runs/{project_name[m]}',
                    name=f'{base_model}{s}{models[m]}_k{k}_{e.strip(".")}',
                    plots=True,  # Enable training plots
                    val=True,     # Enable validation for metrics
                    #fraction = 0.1,  # Use 10% of the data for testing                )
                )

100%|██████████| 50.7M/50.7M [00:01<00:00, 29.5MB/s]


New https://pypi.org/project/ultralytics/8.3.164 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.155 🚀 Python-3.9.21 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1050, 1992MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/training/config_yamls/fold_0_pose.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11-posel_k0_pt_

/home/ce_guest/chess-position-detection/venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 8306.1±1214.9 MB/s, size: 1475.0 KB)


train: Scanning /home/ce_guest/chess-position-detection/dataset/training/pose/fold_1/labels... 2250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2250/2250 [00:08<00:00, 267.58it/s]

...

        3/3      1.39G     0.4804     0.3463          0     0.3753      1.064          4        640:  13%|█▎        | 289/2250 [03:30<23:50,  1.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 375/375 [01:09<00:00,  5.42it/s]

                   all        750        750          1          1      0.995      0.928          1          1      0.995      0.986



3 epochs completed in 1.027 hours.
Optimizer stripped from runs/board/yolo11-posel_k0_pt_3000_2/weights/last.pt, 52.9MB
Optimizer stripped from runs/board/yolo11-posel_k0_pt_3000_2/weights/best.pt, 52.9MB

Validating runs/board/yolo11-posel_k0_pt_3000_2/weights/best.pt...
Ultralytics 8.3.155 🚀 Python-3.9.21 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1050, 1992MiB)
YOLO11l-pose summary (fused): 199 layers, 26,129,899 parameters, 0 gradients, 90.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 375/375 [01:08<00:00,  5.47it/s]


                   all        750        750          1          1      0.995       0.96          1          1      0.995       0.98
Speed: 1.5ms preprocess, 86.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/board/yolo11-posel_k0_pt_3000_2
